# TensorFlow Lite Conversion

In [1]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from models import ArcFaceModel
import tensorflow as tf
import os
import logging
import numpy as np
from pathlib import Path
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model, load_model
from layers import ArcMarginPenaltyLogists
from losses import softmax_loss
from glob import glob
import dataset

tf.get_logger().setLevel(logging.ERROR)
# os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
!echo $CUDA_VISIBLE_DEVICES

1


In [3]:
### IJB-C Dataset

# batch_size = 128
# input_size = 112
# embd_shape = 512
# head_type = 'ArcHead'
# backbone_type = 'MobileNetV2'
# w_decay=5e-4
# num_classes = 3584 
# base_lr = 0.01
# dataset_len = 13033 
# epochs = 100
# steps_per_epoch = dataset_len // batch_size

### MS1M dataset

batch_size = 128 # Initially 128
input_size = 112
embd_shape = 512
head_type = 'ArcHead'
backbone_type = 'EfficientNetLite0' # MobileNetV2, ResNet50, EfficientNetLite0~6
w_decay=5e-4
num_classes = 85742 
dataset_len = 5822653 
base_lr = 0.01 # initially 0.01
epochs = 20
save_steps = 1000
train_size = int(0.8 * dataset_len)
steps_per_epoch = train_size // batch_size
val_size = dataset_len - train_size
validation_steps = val_size // batch_size
steps = 1
is_ccrop=False
binary_img=True
is_Adam = False

dgx = True

In [4]:
from glob import glob
if dgx:
    base_dir = "/raid/workspace/jbpark/FaceRecognition/checkpoints/w_tfidentity/"
    # save_name = "ms1m_mobilenet_check/SGD/*"
    save_name = "ms1m_efficientnet_check/SGD/*"
else:
    base_dir = "/hd/jbpark/models/checkpoints/w_tfidentity/"
    save_name = "ms1m_check/SGD/*"
file_list = []
for files in glob(base_dir+save_name):
    file_list.append(files.split('/')[-1].split('l_')[-1])
file_list.sort()

load_file_name = []
for files in glob(base_dir+save_name):
    if file_list[0] == files.split('/')[-1].split('l_')[-1]:
        load_file_name = files
best_checkpoint = load_file_name.split('.data')[0]
initial_epoch = int(load_file_name.split('e_')[-1].split('_')[0])-1
print(initial_epoch)
print(best_checkpoint)

0
/raid/workspace/jbpark/FaceRecognition/checkpoints/w_tfidentity/ms1m_efficientnet_check/SGD/e_1_l_36.633323669433594.ckpt


In [5]:
accuracy = file_list[0].split('.')[0]
print(accuracy)

36


### Without Quantization

In [6]:
weight_file = best_checkpoint

model = ArcFaceModel(size=input_size,
                         backbone_type=backbone_type,
                         training=False)
model.load_weights(weight_file)

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TF Lite model.
if dgx:
    Path('/raid/workspace/jbpark/FaceRecognition/tflite/').mkdir(parents=True, exist_ok=True)
    with tf.io.gfile.GFile('/raid/workspace/jbpark/FaceRecognition/tflite/ms1m_acc_'+str(accuracy)+'.tflite', 'wb') as f:
        f.write(tflite_model)
else:
    Path('/hd/jbpark/models/tflite/').mkdir(parents=True, exist_ok=True)
    with tf.io.gfile.GFile('/hd/jbpark/models/tflite/ms1m_acc_'+str(accuracy)+'.tflite', 'wb') as f:
        f.write(tflite_model)

### With Qunatization

In [6]:
# train_data_dir = "/raid/workspace/jbpark/IJB-C_Asian/"
# tfrecord_name = train_data_dir+'ijbc_bin.tfrecord'
if dgx:
    train_data_dir = "/raid/workspace/jbpark/ms1m/"
    tfrecord_name = train_data_dir+'ms1m_bin.tfrecord'
else:
    train_data_dir = "/hd/jbpark/dataset/ms1m/"
    tfrecord_name = train_data_dir+'ms1m_bin.tfrecord'


train_dataset = dataset.load_representative_dataset(
    tfrecord_name, batch_size=1, train_size=train_size, binary_img=binary_img,
    is_ccrop=is_ccrop)

In [7]:
def representative_data_gen():
    count=0
    for input_value in train_dataset:
        count+=1
        if count > 100: return
        yield [input_value]

In [8]:
for data in representative_data_gen():
    print(data)

[<tf.Tensor: shape=(1, 112, 112, 3), dtype=float32, numpy=
array([[[[0.72273004, 0.6773497 , 0.6528054 ],
         [0.73559767, 0.68727064, 0.6604309 ],
         [0.75607866, 0.7040435 , 0.6739592 ],
         ...,
         [0.93913627, 0.92783535, 0.8967577 ],
         [0.9419243 , 0.9306233 , 0.8995457 ],
         [0.94272083, 0.9314199 , 0.9003423 ]],

        [[0.7056957 , 0.6603153 , 0.63118005],
         [0.72044754, 0.6708293 , 0.6393986 ],
         [0.7477913 , 0.6944649 , 0.6617982 ],
         ...,
         [0.9418783 , 0.93057734, 0.8994998 ],
         [0.9418783 , 0.93057734, 0.8994998 ],
         [0.9450646 , 0.9337636 , 0.902686  ]],

        [[0.7069825 , 0.6616021 , 0.6324668 ],
         [0.7258091 , 0.67619085, 0.6447601 ],
         [0.7516516 , 0.6983252 , 0.6656585 ],
         ...,
         [0.93844694, 0.92714596, 0.8960684 ],
         [0.93844694, 0.92714596, 0.8960684 ],
         [0.9416332 , 0.93033224, 0.8992547 ]],

        ...,

        [[0.08237889, 0.07139799,

In [9]:
weight_file = best_checkpoint

model = ArcFaceModel(size=input_size,
                         backbone_type=backbone_type,
                         training=False)
model.load_weights(weight_file)

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()

# Save the TF Lite model.
if dgx:
    Path('/raid/workspace/jbpark/FaceRecognition/tflite/').mkdir(parents=True, exist_ok=True)
    with tf.io.gfile.GFile('/raid/workspace/jbpark/FaceRecognition/tflite/ms1m_acc_'+str(accuracy)+'.tflite', 'wb') as f:
        f.write(tflite_model)
else:
    Path('/hd/jbpark/models/tflite/').mkdir(parents=True, exist_ok=True)
    with tf.io.gfile.GFile('/hd/jbpark/models/tflite/ms1m_acc_'+str(accuracy)+'.tflite', 'wb') as f:
        f.write(tflite_model)

RuntimeError: Quantization not yet supported for op: 'DIV'.


### Input / Output Check

In [7]:
import numpy as np
import tensorflow as tf

# Load the TFLite model and allocate tensors.
if dgx:
    interpreter = tf.lite.Interpreter(model_path='/raid/workspace/jbpark/FaceRecognition/tflite/ms1m_acc_'+str(accuracy)+'.tflite')
else:
    interpreter = tf.lite.Interpreter(model_path='/hd/jbpark/models/tflite/ms1m_acc_'+str(accuracy)+'.tflite')
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
input_details

[{'name': 'input_image',
  'index': 0,
  'shape': array([  1, 112, 112,   3], dtype=int32),
  'shape_signature': array([ -1, 112, 112,   3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [8]:
output_details = interpreter.get_output_details()
output_details

[{'name': 'Identity',
  'index': 180,
  'shape': array([  1, 512], dtype=int32),
  'shape_signature': array([ -1, 512], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]